In [1]:
import sys
import os
import time

sys.path.insert(0, '..')
sys.path.insert(0, '../../futile/python')

package = 'YamlDict'

import BigDFT
BigDFT.Logger.path = os.path.join(os.getcwd(), f'BigDFT_test{package}.yaml')
BigDFT.Logger.level = 'debug'
BigDFT.Logger.overwrite = True
BigDFT.Logger.info(f'{package} unit testing begins')

In [2]:
from BigDFT.YamlDict import YamlDict
name = 'test.yaml'

def dict_to_str(d):
    if len(d) == 0:
        return '{}'
    ret = ['{']
    for key in sorted(d.keys()):
        val = d[key]
        
        if isinstance(val, dict):
            val = dict_to_str(val)
        ret.append(f'{key}: {val}, ')
            
    return ''.join(ret)[:-2] + '}'

def wipe(name='test.yaml'):    
    with open(name, 'w') as w:
        w.write('')
        
    return name

dbname = wipe(name)

In [3]:
# NBVAL_CHECK_OUTPUT
# test initialisation from dicttype
wipe()

testyaml = YamlDict({'testvariable': 'test'}, dbname)

print(dict_to_str(testyaml.dict))

{testvariable: test}


In [4]:
# NBVAL_CHECK_OUTPUT
# check value transmission across dicts
wipe()

sender = YamlDict({'testing a sent value':'this is a test'}, dbname)

receiver = YamlDict(fname=dbname)

print(dict_to_str(receiver.dict))

{testing a sent value: this is a test}


In [5]:
# NBVAL_CHECK_OUTPUT
# test nested values, deletion and assignment

nested = YamlDict({'a': 1}, dbname)

nested['b'] = 2
nested['a'] = {'c': 3}
nested['delete_me'] = 4
del nested['delete_me']

print(dict_to_str(nested))

{a: {c: 3}, b: 2, testing a sent value: this is a test}


In [6]:
# NBVAL_CHECK_OUTPUT
# equality checking
wipe()

a = YamlDict({'a': 4}, dbname)
b = YamlDict({'a': 5}, dbname)
c = YamlDict(fname=dbname)
c['a'] = 4

print(a == b)
print(a == c)

False
True


In [7]:
# NBVAL_CHECK_OUTPUT
# create a yaml file and initialise from it
# also tests that lists behave
wipe()

yamlfile = """'test_list':
    - 1
    - 2
    - 3
'test_str': 'test'
"""

with open('fromyaml.yaml', 'w+') as o:
    o.write(yamlfile)
    
fromfile = YamlDict(fname='fromyaml.yaml')

print(dict_to_str(fromfile.dict))
print(type(fromfile.dict['test_list']))

{test_list: [1, 2, 3], test_str: test}
<class 'list'>


In [8]:
# NBVAL_CHECK_OUTPUT
# check naming enforcement
testcases = ['test',
             'test.case',
             'test.txt',
             'test.yml',
             'test.case.txt']

for case in testcases:
    corrected = YamlDict._validate_name(case)
    
    # need to remove lower level directories
    print(corrected.split('/')[-1])

test.yaml
test.yaml
test.yaml
test.yml
test.case.yaml


In [9]:
# NBVAL_CHECK_OUTPUT
# initialising on top of a file with initial dict should merge, rather than overwrite
wipe()

a = YamlDict({'initial': 'should be present'}, fname=dbname)
print(dict_to_str(a.dict))

b = YamlDict({'newval': 'shouldnt overwrite'}, fname=dbname)
print(dict_to_str(b.dict))

# now force overwrite
c = YamlDict({'cleaned': 'value'}, fname=dbname, overwrite=True)
print(dict_to_str(c.dict))

# overwrite with empty dict
d = YamlDict(fname=dbname, overwrite=True)
print(dict_to_str(d.dict))

{initial: should be present}
{initial: should be present, newval: shouldnt overwrite}
{cleaned: value}
{}


In [10]:
# NBVAL_CHECK_OUTPUT
# test updating a linked dict
wipe()

# create linked dicts
a = YamlDict({'initval': 1}, fname=dbname)
b = YamlDict(fname=dbname)

print(dict_to_str(b.dict))
a['newval'] = 2  # set a new val in dict a
# verify that it can be accessed in dict b
print('newval = ', b['newval'])
print(dict_to_str(b.dict))

{initval: 1}
newval =  2
{initval: 1, newval: 2}
